## 初始状态项目结构
```
project
├── code
|   └── e2d_deepfasion.ipynb
└── dataset
    └── deepfasion-multimodel
        ├── images
        ├── test_captions.json
        └── train_captions.json
```

如果不按照此代码框架运行，可能会出问题，或者也可以根据你自己的文件分布，更改`ipynb`代码。

我们首先进行数据集的处理工作。

## 数据集处理方案

数据集下载完成后，我们需要对其进行处理，以适合之后构造的PyTorch数据集类读取。

- 对于文本描述，我们首先构建词典，然后根据词典将文本描述转化为向量。
- 对于图像，我们这里仅记录文件路径。在模型训练与测试阶段，再进行读取。

In [3]:
import os
import json
import random

import torch

code_running_path = os.getcwd()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f"The code is running in path: `{code_running_path}`")
print(f"Device of your machine: {device}")

base_path = "../dataset/deepfashion-multimodal/"

dataset_path = {
  "images": os.path.join(base_path, "images"),
  "train_label": os.path.join(base_path, "train_captions.json"),
  "test_label": os.path.join(base_path, "test_captions.json")
}


def process_dataset(min_word_count: int = 5):
    with open(dataset_path["train_label"]) as f:
        train_labels = json.load(f)
    with open(dataset_path["test_label"]) as f:
        test_labels = json.load(f)
    train_labels_name = list(train_labels.keys())
    
    # randomly choose 20% of the training data as validation data
    random.shuffle(train_labels_name)
    val_labels_name = train_labels_name[:int(len(train_labels_name) * 0.2)]
    train_labels_name = train_labels_name[int(len(train_labels_name) * 0.2):]
    test_labels_name = list(test_labels.keys())
    
process_dataset()


The code is running in path: `/d_workspace/Python/NNDL/code`
Device of your machine: cuda:0
Device name: NVIDIA GeForce RTX 4070 Laptop GPU
Device capability: 8.9
Device memory: 7.99560546875 GB
Device multi-processor count: 36


AttributeError: 'torch._C._CudaDeviceProperties' object has no attribute 'l2_cache_size'